In [1]:
# instalar as bibliotecas:
# pip install stable-baselines3[extra]
# pip install lib
# pip install kaggle-environments
# pip install tf-slim


import gym
from gym import spaces

import numpy.random as rnd
import numpy as np
import random
import tensorflow as tf
import tf_slim as slim 

import matplotlib as mpl
import matplotlib.pyplot as plt
from tensorflow import keras
from collections import deque 
from sklearn.preprocessing import label_binarize

from stable_baselines3.common.env_checker import check_env

In [2]:
state = label_binarize([0, 1, 2, 3], classes=[0, 1, 2, 3])
print("Matriz dos Estados:")
print(state)

Matriz dos Estados:
[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]


# Ambiente Gym Customizado
https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/master/5_custom_gym_env.ipynb#scrollTo=rYzDXA9vJfz1

In [3]:
# Creating the custom environment: ChannelOccupancy 

class ChannelOccupancyEnv(gym.Env):
    
    def __init__(self):
        super(ChannelOccupancyEnv, self).__init__()
        
        # Filled in reset function
        self.action_space = None
        self.observation_space = None
        self.full_states = label_binarize([0, 1, 2, 3], classes=[0, 1, 2, 3])
        self.state = None
        self.steps_beyond_done = None
        done = False

        # Define action and observation space
        # They must be gym.spaces objects
    
        self.action_space = spaces.Discrete(4)
    
        # The observation will be the coordinate of the agent
        # this can be described both by Discrete and Box space
        self.observation_space = spaces.Box(low=0.0, high=1.0, shape=(4,), dtype=np.float32)
    
    def reset(self):
        self.state = self.full_states[0,:]
        return self.state 
    
    def step(self, action):
        done = False
        M= 1
        for m in range(M):
            #print('this self',self.state)
            #self.state = self.full_states[0,:] #self.full_states[m % 4,:]
            #print(self.full_states[0] == self.state)
            
            # Como inicialmente o State já se encontra na posição de indice 0, então encontra o index do ele não nulo
            currentStateIndex = np.argwhere(self.state == 1)
            
            # partir do indice anterior add +1 para a posicção seguinte da matriz de estados
            nextStateIndex = (currentStateIndex[0][0] + 1) % 4
            
            if action == 0 or action == 1 or action ==2 or action == 3:
                
                #self.next_state = self.full_states[(m + 1) % 4,:]
                self.next_state = self.full_states[nextStateIndex]
                
                            
            next_state_idx = np.argwhere(self.next_state==1)[0][0]
            
            if action == next_state_idx:
                reward = -1.0
            else:
                reward = +1.0
            
            self.state = self.next_state
                
               #np.array([self.next_state]).astype(np.float32), reward, {}, {}
        return self.next_state, reward, done, {}
        
        
    def render(self, mode='human'):
        return
    
    def close(self):
        pass

# Validação do Ambiente

In [4]:
env = ChannelOccupancyEnv()
# If the environment don't follow the interface, an error will be thrown
check_env(env, warn=True)

# Testando o Ambiente

In [5]:
env = ChannelOccupancyEnv()

state = env.reset()
epsilon = 0.6
env.render()

print("obs space          :", env.observation_space)
print("action space       :", env.action_space)
#print("action space sample:", env.action_space.sample())

#action = env.action_space.sample()
"""
n_steps = 10
for step in range(n_steps):
    
    action = env.action_space.sample()
    print("Step {}".format(step + 1))    
    next_state, reward, done, info = env.step(action)
    print("estado atual:",state)
    print("next_state  :", next_state)
    print("action      :",action)
    print("reward      :", reward)
    #print('state',state, 'next_state=', next_state, 'reward=', reward, 'done=', done)
    state = next_state

    env.render()
    #if done:
    #    print("reward=", reward)
"""

obs space          : Box(0.0, 1.0, (4,), float32)
action space       : Discrete(4)


'\nn_steps = 10\nfor step in range(n_steps):\n    \n    action = env.action_space.sample()\n    print("Step {}".format(step + 1))    \n    next_state, reward, done, info = env.step(action)\n    print("estado atual:",state)\n    print("next_state  :", next_state)\n    print("action      :",action)\n    print("reward      :", reward)\n    #print(\'state\',state, \'next_state=\', next_state, \'reward=\', reward, \'done=\', done)\n    state = next_state\n\n    env.render()\n    #if done:\n    #    print("reward=", reward)\n'

# Utilizando o ambiente para treinamento com SB3

In [6]:
from stable_baselines3 import DQN # chama a biblioteca que já implementa o algoritmo DQN 
from stable_baselines3.common.cmd_util import make_vec_env


# Instantiate the env
env = ChannelOccupancyEnv()
# create vectorized 
env = make_vec_env(lambda: env, n_envs=1)

/home/sheila/anaconda3/lib/python3.8/site-packages/stable_baselines3/common/cmd_util.py:5: FutureWarning: Module ``common.cmd_util`` has been renamed to ``common.env_util`` and will be removed in the future.
  warnings.warn(


# Alterando a estrutura da rede
por definição a rede já vem configurada com 2 hidden layers de 64 nós, como não estamos usando nenhum hidden layer nesta implementação, então net_arch=[] não recebe nenhum valor.
https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html#custom-network-architecture

In [7]:
import gym
import torch as th

# altera a configuracao da rede neural
#policy_kwargs = dict(activation_fn=th.nn.ReLU,
#                     net_arch=[])

In [8]:
# Create the agent
model = DQN('MlpPolicy', env, verbose=1) #policy_kwargs=policy_kwargs,

Using cpu device


In [9]:
model.policy

DQNPolicy(
  (q_net): QNetwork(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (q_net): Sequential(
      (0): Linear(in_features=4, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): ReLU()
      (4): Linear(in_features=64, out_features=4, bias=True)
    )
  )
  (q_net_target): QNetwork(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (q_net): Sequential(
      (0): Linear(in_features=4, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): ReLU()
      (4): Linear(in_features=64, out_features=4, bias=True)
    )
  )
)

In [10]:
# Train the agent
model.learn(total_timesteps=10000, log_interval=4)

In [11]:
# Test the trained agent
state = env.reset()
n_steps = 20

for step in range(n_steps):
    
    #print("c_state=", state)
    action, _ = model.predict(state, deterministic=True) # a action já vem definida no algoritmo_DQN da SB3
    print("Step {}".format(step + 1), "\n")
    print("Action     =", action[0])
    print("c_state    =", state[0])
    next_state, reward, done, info = env.step(action)
    print('next_state =', next_state[0], 'reward=', reward[0], "\n")
    state = next_state
    
    print("---------------------")
    
    

Step 1 

Action     = 3
c_state    = [1. 0. 0. 0.]
next_state = [0. 1. 0. 0.] reward= 1.0 

---------------------
Step 2 

Action     = 2
c_state    = [0. 1. 0. 0.]
next_state = [0. 0. 1. 0.] reward= -1.0 

---------------------
Step 3 

Action     = 3
c_state    = [0. 0. 1. 0.]
next_state = [0. 0. 0. 1.] reward= -1.0 

---------------------
Step 4 

Action     = 2
c_state    = [0. 0. 0. 1.]
next_state = [1. 0. 0. 0.] reward= 1.0 

---------------------
Step 5 

Action     = 3
c_state    = [1. 0. 0. 0.]
next_state = [0. 1. 0. 0.] reward= 1.0 

---------------------
Step 6 

Action     = 2
c_state    = [0. 1. 0. 0.]
next_state = [0. 0. 1. 0.] reward= -1.0 

---------------------
Step 7 

Action     = 3
c_state    = [0. 0. 1. 0.]
next_state = [0. 0. 0. 1.] reward= -1.0 

---------------------
Step 8 

Action     = 2
c_state    = [0. 0. 0. 1.]
next_state = [1. 0. 0. 0.] reward= 1.0 

---------------------
Step 9 

Action     = 3
c_state    = [1. 0. 0. 0.]
next_state = [0. 1. 0. 0.] reward

In [12]:
state = label_binarize([0, 1, 2, 3], classes=[0, 1, 2, 3])

In [13]:
for idx in range(4):

    c_state = state[idx] 
    print("Estado atual:", c_state)
 
    q = model.predict(c_state, deterministic=True)
    print('Q:',q[0])
    print('-----\n')

Estado atual: [1 0 0 0]
Q: 3
-----

Estado atual: [0 1 0 0]
Q: 2
-----

Estado atual: [0 0 1 0]
Q: 3
-----

Estado atual: [0 0 0 1]
Q: 2
-----



# Estrutura da rede neural quando não há hidden layers

In [11]:
model.policy

DQNPolicy(
  (q_net): QNetwork(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (q_net): Sequential(
      (0): Linear(in_features=4, out_features=4, bias=True)
    )
  )
  (q_net_target): QNetwork(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (q_net): Sequential(
      (0): Linear(in_features=4, out_features=4, bias=True)
    )
  )
)